Copyright 2020 The TensorFlow Hub Authors.

```
@title Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License.
You may obtain a copy of the License at
https://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
```

# Phân loại văn bản với BERT

Hướng dẫn này chứa mã hoàn chỉnh để tinh chỉnh BERT để thực hiện phân tích cảm xúc trên tập dữ liệu đánh giá phim IMDB ở dạng văn bản thuần túy. Ngoài việc đào tạo một mô hình, bạn sẽ học cách tiền xử lý văn bản thành một định dạng thích hợp.

Trong sổ tay này, bạn sẽ:

* Tải tập dữ liệu IMDB

* Tải mô hình BERT từ TensorFlow Hub

* Xây dựng mô hình của riêng bạn bằng cách kết hợp BERT với bộ phân loại

* Đào tạo mô hình của riêng bạn, tinh chỉnh BERT như một phần của điều đó

* Lưu mô hình của bạn và sử dụng nó để phân loại các câu

Nếu bạn mới làm việc với tập dữ liệu IMDB, vui lòng xem [Phân loại văn bản cơ bản](https://www.tensorflow.org/tutorials/keras/text_classification) để biết thêm chi tiết.

## Về BERT

BERT và các kiến ​​trúc mã hóa Transformer khác đã rất thành công trên nhiều nhiệm vụ khác nhau trong NLP (xử lý ngôn ngữ tự nhiên). Họ tính toán các biểu diễn không gian véctơ của ngôn ngữ tự nhiên phù hợp để sử dụng trong các mô hình học sâu. Họ BERT của các mô hình sử dụng kiến ​​trúc bộ mã hóa Transformer để xử lý từng token của văn bản đầu vào trong ngữ cảnh đầy đủ của tất cả các token trước và sau, do đó có tên: **Biểu diễn Thể hiện Mã hóa Hai chiều từ Transformer**.

Các mô hình BERT thường được tiền huấn luyện trên một kho văn bản lớn, sau đó được tinh chỉnh cho các tác vụ cụ thể.

## Cài đặt

In [ ]:
# A dependency of the preprocessing for BERT inputs
!pip install -q -U tensorflow-text

Bạn sẽ sử dụng trình tối ưu hóa AdamW từ [tensorflow/models](https://github.com/tensorflow/models).

In [ ]:
!pip install -q tf-models-official

In [ ]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

## Phân tích cảm xúc

Sổ tay này huấn luyện một mô hình phân tích cảm xúc để phân loại các bài đánh giá phim là *tích cực* hay *tiêu cực*, dựa trên nội dung của bài đánh giá.

Bạn sẽ sử dụng [Tập dữ liệu đánh giá phim lớn](https://ai.stanford.edu/~amaas/data/sentiment/) chứa văn bản của 50.000 bài đánh giá phim từ [Cơ sở dữ liệu phim trên Internet](https://www.imdb.com/).

### Tải về tập dữ liệu IMDB

Hãy tải xuống và giải nén tập dữ liệu, sau đó khám phá cấu trúc thư mục.

In [ ]:
url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'

dataset = tf.keras.utils.get_file('aclImdb_v1.tar.gz', url,
                                  untar=True, cache_dir='.',
                                  cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

train_dir = os.path.join(dataset_dir, 'train')

# remove unused folders to make it easier to load the data
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

Tiếp theo, bạn sẽ sử dụng tiện ích `text_dataset_from_directory` để tạo một `tf.data.Dataset` có nhãn.

Tập dữ liệu IMDB đã được chia thành tập huấn luyện và kiểm tra, nhưng nó thiếu tập xác thực. Hãy tạo một tập xác thực bằng cách tách 80:20 của dữ liệu huấn luyện bằng cách sử dụng đối số `validation_split` bên dưới.

Lưu ý: Khi sử dụng các đối số `validation_split` và `subset`, hãy đảm bảo chỉ định một seed ngẫu nhiên hoặc truyền `shuffle=False`, để tách tập xác thực và huấn luyện không có chồng chéo.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 42

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

class_names = raw_train_ds.class_names
train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)

val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)

val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/test',
    batch_size=batch_size)

test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

Hãy cùng xem qua một vài đánh giá.

In [ ]:
for text_batch, label_batch in train_ds.take(1):
  for i in range(3):
    print(f'Review: {text_batch.numpy()[i]}')
    label = label_batch.numpy()[i]
    print(f'Label : {label} ({class_names[label]})')

### Tải các mô hình từ TensorFlow Hub

Tại đây, bạn có thể chọn mô hình BERT mà bạn sẽ tải từ TensorFlow Hub và tinh chỉnh. Có nhiều mô hình BERT có sẵn.

* [BERT-Base](https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3), [Uncased](https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3) và [bảy mô hình khác](https://tfhub.dev/google/collections/bert/1) với trọng số được huấn luyện do các tác giả BERT ban đầu phát hành.

* [Các BERT nhỏ](https://tfhub.dev/google/collections/bert/1) có cùng kiến ​​trúc chung nhưng ít hơn và/hoặc các khối Transformer nhỏ hơn, cho phép bạn khám phá sự đánh đổi giữa tốc độ, kích thước và chất lượng.

* [ALBERT](https://tfhub.dev/google/collections/albert/1): bốn kích thước khác nhau của "*A Lite BERT*" giúp giảm kích thước mô hình (nhưng không phải thời gian tính toán) bằng cách chia sẻ các tham số giữa các lớp.

* [Các chuyên gia BERT](https://tfhub.dev/google/collections/experts/bert/1): tám mô hình đều có kiến ​​trúc cơ sở BERT nhưng cung cấp sự lựa chọn giữa các miền tiền huấn luyện khác nhau, để điều chỉnh chặt chẽ hơn với nhiệm vụ mục tiêu.

* [Electra](https://tfhub.dev/google/collections/electra/1) có kiến ​​trúc tương tự như BERT (ở ba kích thước khác nhau), nhưng được tiền huấn luyện như một người phân biệt trong một thiết lập giống như *Mạng đối nghịch tạo sinh (GAN)*.

* BERT với Talking-Heads Attention và Gated GELU [[cơ bản](https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1), [lớn](https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_large/1)] có hai cải tiến đối với lõi của kiến ​​trúc Transformer.

Tài liệu mô hình trên TensorFlow Hub có nhiều chi tiết hơn và tham chiếu đến các tài liệu nghiên cứu. Theo các liên kết ở trên hoặc nhấp vào URL [tfhub.dev](http://tfhub.dev/) được in sau khi thực thi ô tiếp theo.

Đề xuất là bắt đầu với BERT nhỏ (với ít tham số hơn) vì chúng nhanh hơn để tinh chỉnh. Nếu bạn thích một mô hình nhỏ nhưng có độ chính xác cao hơn, ALBERT có thể là lựa chọn tiếp theo của bạn. Nếu bạn muốn độ chính xác cao hơn nữa, hãy chọn một trong các kích thước BERT cổ điển hoặc các tinh chỉnh gần đây của chúng như Electra, Talking Heads hoặc một BERT Expert.

Ngoài các mô hình có sẵn bên dưới, có [nhiều phiên bản](https://tfhub.dev/google/collections/transformer_encoders_text/1) của các mô hình lớn hơn và có thể mang lại độ chính xác tốt hơn, nhưng chúng quá lớn để có thể tinh chỉnh trên một GPU. Bạn sẽ có thể làm điều đó trên các nhiệm vụ [Giải quyết các nhiệm vụ GLUE bằng BERT trên một TPU colab](https://www.tensorflow.org/text/tutorials/bert_glue).

Bạn sẽ thấy trong đoạn mã bên dưới rằng việc chuyển đổi URL tfhub.dev là đủ để thử bất kỳ mô hình nào trong số này, vì tất cả sự khác biệt giữa chúng được gói gọn trong SavedModels từ TF Hub.

In [ ]:
#@title Choose a BERT model to fine-tune

bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'  #@param ["bert_en_uncased_L-12_H-768_A-12", "bert_en_cased_L-12_H-768_A-12", "bert_multi_cased_L-12_H-768_A-12", "small_bert/bert_en_uncased_L-2_H-128_A-2", "small_bert/bert_en_uncased_L-2_H-256_A-4", "small_bert/bert_en_uncased_L-2_H-512_A-8", "small_bert/bert_en_uncased_L-2_H-768_A-12", "small_bert/bert_en_uncased_L-4_H-128_A-2", "small_bert/bert_en_uncased_L-4_H-256_A-4", "small_bert/bert_en_uncased_L-4_H-512_A-8", "small_bert/bert_en_uncased_L-4_H-768_A-12", "small_bert/bert_en_uncased_L-6_H-128_A-2", "small_bert/bert_en_uncased_L-6_H-256_A-4", "small_bert/bert_en_uncased_L-6_H-512_A-8", "small_bert/bert_en_uncased_L-6_H-768_A-12", "small_bert/bert_en_uncased_L-8_H-128_A-2", "small_bert/bert_en_uncased_L-8_H-256_A-4", "small_bert/bert_en_uncased_L-8_H-512_A-8", "small_bert/bert_en_uncased_L-8_H-768_A-12", "small_bert/bert_en_uncased_L-10_H-128_A-2", "small_bert/bert_en_uncased_L-10_H-256_A-4", "small_bert/bert_en_uncased_L-10_H-512_A-8", "small_bert/bert_en_uncased_L-10_H-768_A-12", "small_bert/bert_en_uncased_L-12_H-128_A-2", "small_bert/bert_en_uncased_L-12_H-256_A-4", "small_bert/bert_en_uncased_L-12_H-512_A-8", "small_bert/bert_en_uncased_L-12_H-768_A-12", "albert_en_base", "electra_small", "electra_base", "experts_pubmed", "experts_wiki_books", "talking-heads_base"]

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

### Mô hình tiền xử lý

Các đầu vào văn bản cần được chuyển đổi thành token id số và được sắp xếp thành nhiều Tensors trước khi được nhập vào BERT. TensorFlow Hub cung cấp một mô hình tiền xử lý phù hợp cho từng mô hình BERT được thảo luận ở trên, mô hình này thực hiện chuyển đổi này bằng cách sử dụng các hoạt động TF từ thư viện `TF.text`. Không cần thiết phải chạy mã Python thuần túy bên ngoài mô hình TensorFlow của bạn để tiền xử lý văn bản.

Mô hình tiền xử lý phải là mô hình được tham chiếu bởi tài liệu của mô hình BERT, bạn có thể đọc mô hình này tại URL được in ở trên. Đối với các mô hình BERT từ menu thả xuống ở trên, mô hình tiền xử lý được chọn tự động.

Lưu ý: Bạn sẽ tải mô hình tiền xử lý vào một [hub.KerasLayer](https://www.tensorflow.org/hub/api_docs/python/hub/KerasLayer) để soạn mô hình đã tinh chỉnh của bạn. Đây là API ưa tiên để tải SavedModel kiểu TF2 từ TF Hub vào mô hình Keras.

In [ ]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

Hãy thử mô hình tiền xử lý trên một số văn bản và xem kết quả:

In [ ]:
text_test = ['this is such an amazing movie!']
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Như bạn có thể thấy, bây giờ bạn có 3 đầu ra từ tiền xử lý mà mô hình BERT sẽ sử dụng `(input_words_id, input_mask` và `input_type_ids)`.

Một số điểm quan trọng khác:

* Đầu vào được cắt ngắn thành 128 token. Số lượng token có thể được tùy chỉnh và bạn có thể xem thêm chi tiết về [Giải quyết các nhiệm vụ GLUE bằng BERT trên một TPU colab](https://www.tensorflow.org/text/tutorials/bert_glue).

* Các `input_type_ids` chỉ có một giá trị (0) vì đây là một đầu vào câu đơn. Đối với đầu vào nhiều câu, nó sẽ có một số cho mỗi đầu vào.

Vì bộ tiền xử lý văn bản này là một mô hình TensorFlow, nên nó có thể được đưa trực tiếp vào mô hình của bạn.

### Sử dụng mô hình BERT

Trước khi đưa BERT vào mô hình của riêng bạn, chúng ta hãy xem xét kết quả đầu ra của nó. Bạn sẽ tải nó từ TF Hub và xem các giá trị trả về.

In [ ]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [ ]:
bert_results = bert_model(text_preprocessed)

print(f'Loaded BERT: {tfhub_handle_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

Các mô hình BERT trả về một bản đồ với 3 khóa quan trọng: `pooled_output`, `sequence_output`, `encoder_outputs`:

* `pooled_output` đại diện cho toàn bộ chuỗi đầu vào. Hình dạng là `[batch_size, H]`. Bạn có thể coi đây là một phần nhúng cho toàn bộ bài đánh giá phim.

* `sequence_output` đại diện cho mỗi token đầu vào trong ngữ cảnh. Hình dạng là `[batch_size, seq_length, H]`. Bạn có thể coi đây là cách nhúng theo ngữ cảnh cho mỗi token trong bài đánh giá phim.

* `encoder_outputs` là các kích hoạt trung gian của các khối `L` Transformer. `outputs["encoder_outputs"][i]` là Tensor của hình dạng `[batch_size, seq_length, 1024]` với các đầu ra của khối Transformer `thứ i`, cho `0 <= i < L`. Giá trị cuối cùng của danh sách bằng `sequence_output`.

Để tinh chỉnh, bạn sẽ sử dụng mảng `pooled_output`.

### Định nghĩa mô hình của bạn

Bạn sẽ tạo một mô hình được tinh chỉnh rất đơn giản, với mô hình tiền xử lý, mô hình BERT đã chọn, một lớp Dense và một lớp Dropout.

Lưu ý: để biết thêm thông tin về đầu vào và đầu ra của mô hình cơ sở, bạn có thể theo dõi URL của mô hình để làm tài liệu. Cụ thể ở đây, bạn không cần phải lo lắng về điều đó vì mô hình tiền xử lý sẽ đảm nhận việc đó cho bạn.

In [ ]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

Hãy kiểm tra xem mô hình có chạy với đầu ra của mô hình tiền xử lý hay không.

In [ ]:
classifier_model = build_classifier_model()
bert_raw_result = classifier_model(tf.constant(text_test))
print(tf.sigmoid(bert_raw_result))

Tất nhiên, đầu ra là vô nghĩa vì người mô hình chưa được huấn luyện.

Chúng ta hãy nhìn vào cấu trúc của mô hình.

In [ ]:
tf.keras.utils.plot_model(classifier_model)

### Đào tạo người mẫu

Bây giờ bạn có tất cả các phần để huấn luyện một mô hình, bao gồm mô-đun tiền xử lý, bộ mã hóa BERT, dữ liệu và bộ phân loại.

#### Hàm mất mát

Vì đây là một vấn đề phân loại nhị phân và mô hình xuất ra một xác suất (một lớp đơn vị), bạn sẽ sử dụng hàm mất mát `losses.BinaryCrossentropy`.

In [ ]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

#### Trình tối ưu hóa

Để tinh chỉnh, hãy sử dụng cùng một trình tối ưu hóa mà BERT đã được huấn luyện ban đầu: "*Khoảnh khắc thích ứng*" (Adam). Trình tối ưu hóa này giảm thiểu sự mất mát dự đoán và thực hiện điều hòa bằng phân rã trọng số (không sử dụng moment), còn được gọi là [AdamW](https://arxiv.org/abs/1711.05101) .

Đối với tốc độ học (`init_lr`), bạn sẽ sử dụng cùng một lịch trình như tiền huấn luyện BERT: phân rã tuyến tính của tốc độ học ban đầu theo lý thuyết, tiền tố với giai đoạn khởi động tuyến tính trong 10% bước huấn luyện đầu tiên (`num_warmup_steps`). Phù hợp với bài báo BERT, tốc độ học ban đầu nhỏ hơn để tinh chỉnh (tốt nhất là 5e-5, 3e-5, 2e-5).

In [ ]:
epochs = 5
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

#### Tải mô hình BERT và huấn luyện

Sử dụng `classifier_model` mà bạn đã tạo trước đó, bạn có thể biên dịch mô hình với mất mát, chỉ số đo lường và trình tối ưu hóa.

In [ ]:
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

Lưu ý: thời gian huấn luyện sẽ thay đổi tùy thuộc vào độ phức tạp của mô hình BERT bạn đã chọn.

In [ ]:
print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(x=train_ds,
                               validation_data=val_ds,
                               epochs=epochs)

#### Đánh giá mô hình

Hãy xem mô hình hoạt động như thế nào. Hai giá trị sẽ được trả về. Mất mát (một số đại diện cho lỗi, các giá trị càng thấp thì càng tốt) và độ chính xác.

In [ ]:
loss, accuracy = classifier_model.evaluate(test_ds)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

#### Vẽ biểu đồ độ chính xác và mất mát theo thời gian

Dựa trên đối tượng `History` trả về bởi `model.fit()`. Bạn có thể vẽ biểu đồ của quá trình huẩn luyện và mất mát xác thực để so sánh, cũng như độ chính xác của quá trình huấn luyện và xác thực:

In [ ]:
history_dict = history.history
print(history_dict.keys())

acc = history_dict['binary_accuracy']
val_acc = history_dict['val_binary_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)
fig = plt.figure(figsize=(10, 6))
fig.tight_layout()

plt.subplot(2, 1, 1)
# "bo" is for "blue dot"
plt.plot(epochs, loss, 'r', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
# plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

Trong biểu đồ này, các đường màu đỏ thể hiện sự mất mát trong quá trình huấn luyện và độ chính xác, còn các đường màu xanh là sự mất mát xác thực và độ chính xác.

### Xuất dùng để suy luận

Bây giờ bạn chỉ cần lưu mô hình đã tinh chỉnh của mình để sử dụng sau này.

In [ ]:
dataset_name = 'imdb'
saved_model_path = './{}_bert'.format(dataset_name.replace('/', '_'))

classifier_model.save(saved_model_path, include_optimizer=False)

Hãy tải lại mô hình, để bạn có thể thử song song với mô hình vẫn còn trong bộ nhớ.

In [ ]:
reloaded_model = tf.saved_model.load(saved_model_path)

Tại đây, bạn có thể kiểm tra mô hình của mình trên bất kỳ câu nào bạn muốn, chỉ cần thêm vào biến ví dụ bên dưới.

In [ ]:
def print_my_examples(inputs, results):
  result_for_printing = \
    [f'input: {inputs[i]:<30} : score: {results[i][0]:.6f}'
                         for i in range(len(inputs))]
  print(*result_for_printing, sep='\n')
  print()


examples = [
    'this is such an amazing movie!',  # this is the same sentence tried earlier
    'The movie was great!',
    'The movie was meh.',
    'The movie was okish.',
    'The movie was terrible...'
]

reloaded_results = tf.sigmoid(reloaded_model(tf.constant(examples)))
original_results = tf.sigmoid(classifier_model(tf.constant(examples)))

print('Results from the saved model:')
print_my_examples(examples, reloaded_results)
print('Results from the model in memory:')
print_my_examples(examples, original_results)

Nếu bạn muốn sử dụng mô hình của mình trên [TF Serving](https://www.tensorflow.org/tfx/guide/serving), hãy nhớ rằng nó sẽ gọi `SavedModel` của bạn thông qua một trong các chữ ký được đặt tên của nó. Trong Python, bạn có thể kiểm tra chúng như sau:

In [ ]:
serving_results = reloaded_model \
            .signatures['serving_default'](tf.constant(examples))

serving_results = tf.sigmoid(serving_results['classifier'])

print_my_examples(examples, serving_results)